- 입력: mp4 (또는 webcam)
- 처리: YOLOv8-pose → (frame feature 85) → 최근 T=40 프레임 LSTM 추론
- 출력: mp4 (확률 오버레이 포함)


## 1) Imports & 경로/파라미터

In [132]:
from ultralytics import YOLO
import cv2
import numpy as np
import torch
import torch.nn as nn
from collections import deque
from pathlib import Path

In [133]:
# ===== 경로 =====
DATA_ROOT = Path("../data")
MODEL_PATH = Path("./best_lstm.pt")          # 02_train_lstm.ipynb에서 저장한 파일
YOLO_POSE_WEIGHTS = "yolov8n-pose.pt"        # 또는 본인 가중치 경로

# 입력/출력 영상
INPUT_VIDEO = str(DATA_ROOT / "train" / "raw" /"video" / "N" / "N" / "00010_H_A_N_C4" / "00010_H_A_N_C4.mp4")
OUTPUT_VIDEO = "out/00010_H_A_N_C4_overlay.mp4"

# ===== 학습과 동일해야 하는 파라미터 =====
NUM_FRAMES = 100          # (학습에서 고정)
T = 40                    # sequence length (4초 @10fps)
INPUT_DIM = 85            # feature dim
FALL_CLASS_INDEX = 1      # softmax에서 낙상 클래스 인덱스

# ===== 추론 설정 =====
TARGET_FPS = 10           # 학습 fps에 맞추기 위해 프레임 샘플링 (입력 fps가 달라도 10fps로 처리)
EMA_ALPHA = 0.2           # 확률 스무딩 (0~1) - 클수록 즉시 반응
SHOW_SKELETON = True      # keypoint 표시 여부

## 2) LSTM 모델 정의 & 로드

In [134]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

DEVICE: cpu


In [135]:
class FallLSTM(nn.Module):
    def __init__(self, input_dim=85, hidden_dim=128, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.3
        )
        self.fc = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

In [136]:
model = FallLSTM(input_dim=INPUT_DIM).to(DEVICE)
state = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(state)
model.eval()

print("loaded:", MODEL_PATH)

loaded: best_lstm.pt


/var/folders/1w/f97855ms55x_bvptq4cmnmmh0000gn/T/ipykernel_73914/103494679.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(MODEL_PATH, map_location=DE

## 3) YOLOv8 Pose 로드

In [137]:
pose_model = YOLO(YOLO_POSE_WEIGHTS)
print("loaded yolo:", YOLO_POSE_WEIGHTS)

loaded yolo: yolov8n-pose.pt


## 4) 학습과 동일한 전처리: frame → feature(85)


In [138]:
def kpts_to_feature(xy: np.ndarray, conf: np.ndarray, prev_xy_norm: np.ndarray):
    """
    xy: (17,2) raw pixel
    conf: (17,)
    prev_xy_norm: (17,2) or None  (이전 프레임 정규화 xy)

    return:
      feat: (85,)
      xy_norm: (17,2)  (다음 프레임 velocity 계산용)
    """
    xy = xy.astype(np.float32)
    conf = conf.astype(np.float32)

    # hip center (11,12)
    center = (xy[11] + xy[12]) / 2.0
    xy_c = xy - center

    # scale (shoulder-hip) with epsilon
    scale = np.linalg.norm(xy_c[5] - xy_c[11]) + 1e-6
    xy_n = xy_c / scale

    if prev_xy_norm is None:
        vel = np.zeros_like(xy_n, dtype=np.float32)
    else:
        vel = (xy_n - prev_xy_norm).astype(np.float32)

    feat = np.concatenate([xy_n.flatten(), conf, vel.flatten()]).astype(np.float32)  # 34+17+34=85
    return feat, xy_n

## 5) YOLO 포즈 추출: frame → (xy, conf) or zeros

In [139]:
def extract_pose_one(frame_bgr: np.ndarray):
    """
    return:
      xy: (17,2)
      conf: (17,)
      has_person: bool
    """
    res = pose_model(frame_bgr, verbose=False)[0]

    if (res.keypoints is None) or (res.keypoints.xy is None) or (len(res.keypoints.xy) == 0):
        xy = np.zeros((17,2), dtype=np.float32)
        conf = np.zeros((17,), dtype=np.float32)
        return xy, conf, False

    # 첫 번째 사람만 사용 (필요 시: 가장 큰 bbox 기준 등으로 개선 가능)
    xy = res.keypoints.xy[0].detach().cpu().numpy().astype(np.float32)      # (17,2)
    conf = res.keypoints.conf[0].detach().cpu().numpy().astype(np.float32)  # (17,)
    return xy, conf, True

## 6) 오버레이 유틸 (텍스트 + 바)

In [140]:
def draw_overlay(frame, p_fall, p_norm, x=20, y=40):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, f"Fall: {p_fall:.2f}", (x, y), font, 0.9, (0,0,255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Normal: {p_norm:.2f}", (x, y+35), font, 0.9, (0,255,0), 2, cv2.LINE_AA)

    # bar
    bar_w, bar_h = 240, 18
    bx, by = x, y+60
    cv2.rectangle(frame, (bx, by), (bx+bar_w, by+bar_h), (255,255,255), 2)
    fill = int(bar_w * float(p_fall))
    cv2.rectangle(frame, (bx, by), (bx+fill, by+bar_h), (0,0,255), -1)
    return frame

def draw_keypoints(frame, xy, conf, thr=0.3):
    for i in range(17):
        if conf[i] >= thr:
            cx, cy = int(xy[i,0]), int(xy[i,1])
            cv2.circle(frame, (cx,cy), 3, (255,255,0), -1)
    return frame

## 7) 비디오 추론 + 저장 (mp4)

- 입력 영상 fps가 10이 아니면 TARGET_FPS=10에 맞춰 샘플링해서 처리합니다.
- LSTM은 최근 T 프레임 feature가 쌓여야 추론을 시작합니다.
- 확률은 EMA로 스무딩해서 흔들림을 줄입니다.

In [141]:
cap = cv2.VideoCapture(INPUT_VIDEO)
if not cap.isOpened():
    raise FileNotFoundError(f"cannot open: {INPUT_VIDEO}")

in_fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print("input fps:", in_fps, "size:", (w,h))

input fps: 10.0 size: (1280, 720)


In [142]:
# output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, float(in_fps if in_fps > 1 else TARGET_FPS), (w, h))

# streaming buffers
feat_buf = deque(maxlen=T)
prev_xy_norm = None
p_fall_ema = 0.0

# fps sampling
if in_fps and in_fps > 1:
    step = max(1, int(round(in_fps / TARGET_FPS)))
else:
    step = 1

frame_idx = 0
written = 0

with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1

        # 샘플링: TARGET_FPS에 맞춰 처리
        if frame_idx % step != 0:
            out.write(frame)
            written += 1
            continue

        xy, conf, has_person = extract_pose_one(frame)

        if SHOW_SKELETON:
            frame = draw_keypoints(frame, xy, conf, thr=0.3)

        # feature 생성
        feat, prev_xy_norm = kpts_to_feature(xy, conf, prev_xy_norm)
        feat_buf.append(feat)

        # 기본값 (버퍼가 차기 전)
        p_fall = p_fall_ema
        p_norm = 1.0 - p_fall_ema

        # 추론 (T개 쌓이면)
        if len(feat_buf) == T:
            x_seq = np.stack(list(feat_buf), axis=0)[None, :, :]  # (1,T,85)
            x_t = torch.tensor(x_seq, dtype=torch.float32, device=DEVICE)
            logits = model(x_t)
            prob = torch.softmax(logits, dim=1).detach().cpu().numpy()[0]

            p_fall_raw = float(prob[FALL_CLASS_INDEX])
            # EMA smoothing
            p_fall_ema = EMA_ALPHA * p_fall_raw + (1 - EMA_ALPHA) * p_fall_ema

            p_fall = p_fall_ema
            p_norm = 1.0 - p_fall_ema

        # 오버레이
        frame = draw_overlay(frame, p_fall, p_norm)

        out.write(frame)
        written += 1

cap.release()
out.release()

print("saved:", OUTPUT_VIDEO, "frames:", written)

saved: out/00010_H_A_N_C4_overlay.mp4 frames: 100
